In [1]:
import cv2
import numpy as np
import pickle


def select_parking_spots(img_path):
    
    
    parking_spots = []
    
    
    drawing = False 
    start_point = None 
    end_point = None  
    
    
    img = cv2.imread(img_path)
    if img is None:
        print(f"HATA: '{img_path}' dosyası bulunamadı!")
        return []
    
    
    def mouse_callback(event, x, y, flags, param):
        nonlocal parking_spots, drawing, start_point, end_point
        
        
        if event == cv2.EVENT_LBUTTONDOWN:
            drawing = True
            start_point = (x, y)
            end_point = (x, y)
            
        
        elif event == cv2.EVENT_MOUSEMOVE:
            if drawing:
                end_point = (x, y)
                
    
        elif event == cv2.EVENT_LBUTTONUP:
            if drawing:
                drawing = False
                end_point = (x, y)
                
                
                x1, y1 = start_point
                x2, y2 = end_point
                
                
                top_left_x = min(x1, x2)
                top_left_y = min(y1, y2)
                width = abs(x2 - x1)
                height = abs(y2 - y1)
                
                if width > 10 and height > 10:
                    parking_spots.append((top_left_x, top_left_y, width, height))
                    print(f"✓ Park yeri eklendi: #{len(parking_spots)} - Konum:({top_left_x},{top_left_y}) Boyut:[{width}x{height}]")
                else:
                    print("⚠ Alan çok küçük! En az 10x10 piksel olmalı.")
                
                
                start_point = None
                end_point = None
        
        elif event == cv2.EVENT_RBUTTONDOWN:
            if parking_spots:
                removed = parking_spots.pop()
                print(f"✗ Park yeri silindi. Kalan: {len(parking_spots)}")
    

    cv2.namedWindow("Park Yeri Secimi - Surukle-Birak", cv2.WINDOW_NORMAL)
    cv2.setMouseCallback("Park Yeri Secimi - Surukle-Birak", mouse_callback)
    
    print("\n" + "="*70)
    print("DİNAMİK PARK YERİ SEÇIM MODU - SÜRÜKLE-BIRAK")
    print("="*70)
    print("KONTROLLER:")
    print("  [Sol Tık + Sürükle]: Park yerini istediğiniz boyutta çizin")
    print("  [Sağ Tık]: Son eklenen park yerini sil")
    print("  [R] Tuşu: Tüm seçimleri sıfırla")
    print("  [Q] Tuşu: Bitir ve kaydet")
    print("="*70)
    print("\nÖNEMLİ: Her park yeri için sol üst köşeden sağ alt köşeye doğru")
    print("        fareyi sürükleyerek tam alanı kaplayacak şekilde çizin!\n")
    

    while True:
        img_copy = img.copy()
        

        for i, (x, y, w, h) in enumerate(parking_spots):

            cv2.rectangle(img_copy, (x, y), (x + w, y + h), (0, 255, 0), 2)
            

            cv2.putText(img_copy, f"{i+1}", (x+3, y+15), 
                       cv2.FONT_HERSHEY_SIMPLEX, 0.4, (255, 255, 255), 1)
            

            cv2.putText(img_copy, f"{w}x{h}", (x+w-40, y+h-3), 
                       cv2.FONT_HERSHEY_SIMPLEX, 0.3, (200, 200, 200), 1)
        

        if drawing and start_point and end_point:
            x1, y1 = start_point
            x2, y2 = end_point
            

            cv2.rectangle(img_copy, start_point, end_point, (0, 255, 255), 2)
            

            width = abs(x2 - x1)
            height = abs(y2 - y1)
            mid_x = (x1 + x2) // 2
            mid_y = (y1 + y2) // 2
            cv2.putText(img_copy, f"{width}x{height}", (mid_x-25, mid_y), 
                       cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 255), 2)
        

        panel_height = 35
        img_width = img_copy.shape[1]
        

        cv2.rectangle(img_copy, (0, 0), (img_width, panel_height), (0, 0, 0), -1)
        

        x_pos = 10
        cv2.putText(img_copy, f"Toplam Park Yeri: {len(parking_spots)}", (x_pos, 22), 
                   cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)
        
        x_pos += 220
        cv2.putText(img_copy, "| Sol Tik+Surukle: Alan Sec | Sag Tik: Sil | Q: Kaydet | R: Sifirla", (x_pos, 22), 
                   cv2.FONT_HERSHEY_SIMPLEX, 0.4, (180, 180, 180), 1)
        

        cv2.imshow("Park Yeri Secimi - Surukle-Birak", img_copy)
        

        key = cv2.waitKey(1) & 0xFF
        

        if key == ord('q') or key == ord('Q'):
            break
        

        elif key == ord('r') or key == ord('R'):
            if parking_spots:
                parking_spots.clear()
                print("⚠ Tüm park yerleri silindi!")
    

    cv2.destroyAllWindows()
    

    if len(parking_spots) > 0:
        with open('parking_spots.pkl', 'wb') as f:
            pickle.dump(parking_spots, f)
        
        print("\n" + "="*70)
        print("KAYIT BAŞARILI!")
        print("="*70)
        print(f"✓ Toplam {len(parking_spots)} park yeri kaydedildi")
        

        if parking_spots:
            widths = [spot[2] for spot in parking_spots]
            heights = [spot[3] for spot in parking_spots]
            areas = [spot[2] * spot[3] for spot in parking_spots]
            
            print(f"\nBoyut İstatistikleri:")
            print(f"  • Genişlik: Min={min(widths)}px, Max={max(widths)}px, Ortalama={sum(widths)//len(widths)}px")
            print(f"  • Yükseklik: Min={min(heights)}px, Max={max(heights)}px, Ortalama={sum(heights)//len(heights)}px")
            print(f"  • Alan: Min={min(areas)}px², Max={max(areas)}px², Ortalama={sum(areas)//len(areas)}px²")
        
        print("="*70 + "\n")
    else:
        print("\n⚠ Hiç park yeri seçilmedi!\n")
    
    return parking_spots


def check_parking_spot(img_processed, x, y, w, h):
  
    spot = img_processed[y:y+h, x:x+w]
    
    if spot.size == 0:
        return False
    
    edges = cv2.Canny(spot, 50, 200)
    edge_ratio = cv2.countNonZero(edges) / (spot.shape[0] * spot.shape[1])
    
    std_dev = np.std(spot)
    
    mean_val = np.mean(spot)
    
    area = w * h
    
    if area < 500: 
        edge_threshold = 0.10
        std_threshold = 15
        mean_threshold = 190
    elif area < 1500: 
        edge_threshold = 0.15
        std_threshold = 20
        mean_threshold = 180
    else:
        edge_threshold = 0.18
        std_threshold = 25
        mean_threshold = 175
    
    
    is_occupied = (edge_ratio > edge_threshold) or (std_dev > std_threshold and mean_val < mean_threshold)
    
    return is_occupied


def analyze_parking(img_path, parking_spots):
    
    img = cv2.imread(img_path)
    if img is None:
        print(f"HATA: '{img_path}' dosyası bulunamadı!")
        return None, 0, 0
    
    img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
    img_enhanced = clahe.apply(img_gray)
    
    img_processed = cv2.GaussianBlur(img_enhanced, (3, 3), 0)
    
    empty_count = 0
    occupied_count = 0
    
    for i, (x, y, w, h) in enumerate(parking_spots):
    
        is_occupied = check_parking_spot(img_processed, x, y, w, h)
        
        if is_occupied:
            color = (0, 0, 255)      
            text = "DOLU"
            occupied_count += 1
        else:
            color = (0, 255, 0)      
            text = "BOS"
            empty_count += 1
        

        cv2.rectangle(img, (x, y), (x + w, y + h), color, 2)
        
        cv2.putText(img, f"{i+1}", (x+3, y+15), 
                   cv2.FONT_HERSHEY_SIMPLEX, 0.4, (255, 255, 255), 1)
        
        cv2.putText(img, text, (x+3, y+h-5), 
                   cv2.FONT_HERSHEY_SIMPLEX, 0.35, color, 1)
    

    total = len(parking_spots)
    img_width = img.shape[1]
    panel_height = 35
    

    cv2.rectangle(img, (0, 0), (img_width, panel_height), (0, 0, 0), -1)
    
    x_pos = 10
    cv2.putText(img, "RAPOR:", (x_pos, 22), 
               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)
    
    x_pos += 85
    cv2.putText(img, f"Toplam: {total}", (x_pos, 22), 
               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (200, 200, 200), 1)
    
    x_pos += 110
    cv2.putText(img, f"Bos: {empty_count}", (x_pos, 22), 
               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1)
    
    x_pos += 90
    cv2.putText(img, f"Dolu: {occupied_count}", (x_pos, 22), 
               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1)
    
    # Doluluk oranı
    doluluk_orani = (occupied_count / total * 100) if total > 0 else 0
    x_pos += 110
    cv2.putText(img, f"Doluluk Orani: %{doluluk_orani:.1f}", (x_pos, 22), 
               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 0), 1)
    
    return img, empty_count, occupied_count


if __name__ == "__main__":

    img_path = "otopark.jpg"
    
    print("\n" + "="*70)
    print(" "*15 + "OTOPARK DOLULUK ANALİZ SİSTEMİ")
    print(" "*18 + "SÜRÜKLE-BIRAK SİSTEMİ")
    print("="*70)
    print("\nÖZELLİKLER:")
    print("  ✓ SÜRÜKLE-BIRAK ile park yeri seçimi")
    print("  ✓ Her alan için ÖZEL BOYUT belirleme")
    print("  ✓ Küçük/büyük tüm park yerlerine uyumlu")
    print("  ✓ Adaptif doluluk tespiti")
    print("  ✓ Kompakt görsel raporlama")
    print("="*70 + "\n")
    
    parking_spots = select_parking_spots(img_path)
    
    if len(parking_spots) == 0:
        print("Program sonlandırılıyor...")
        exit()
    
    print("\n" + "="*70)
    print("ANALİZ YAPILIYOR...")
    print("="*70)
    
    result_img, empty, occupied = analyze_parking(img_path, parking_spots)
    
    if result_img is None:
        print("Analiz başarısız!")
        exit()
    
    cv2.namedWindow("Otopark Doluluk Analizi", cv2.WINDOW_NORMAL)
    cv2.imshow("Otopark Doluluk Analizi", result_img)
    
    total = len(parking_spots)
    
    print("\n" + "="*70)
    print("DETAYLI SONUÇLAR")
    print("="*70)
    print(f"Toplam Park Yeri    : {total}")
    print(f"Boş Park Yerleri    : {empty} (%{empty/total*100:.1f})")
    print(f"Dolu Park Yerleri   : {occupied} (%{occupied/total*100:.1f})")
    print("-"*70)
    print(f"Doluluk Oranı       : %{occupied/total*100:.1f}")
    print("="*70)
    print("\n[ESC] veya herhangi bir tuşa basarak pencereyi kapatabilirsiniz...")
    
    # Bekleme
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    
    print("\nProgram başarıyla tamamlandı!\n")




               OTOPARK DOLULUK ANALİZ SİSTEMİ
                  SÜRÜKLE-BIRAK SİSTEMİ

ÖZELLİKLER:
  ✓ SÜRÜKLE-BIRAK ile park yeri seçimi
  ✓ Her alan için ÖZEL BOYUT belirleme
  ✓ Küçük/büyük tüm park yerlerine uyumlu
  ✓ Adaptif doluluk tespiti
  ✓ Kompakt görsel raporlama


DİNAMİK PARK YERİ SEÇIM MODU - SÜRÜKLE-BIRAK
KONTROLLER:
  [Sol Tık + Sürükle]: Park yerini istediğiniz boyutta çizin
  [Sağ Tık]: Son eklenen park yerini sil
  [R] Tuşu: Tüm seçimleri sıfırla
  [Q] Tuşu: Bitir ve kaydet

ÖNEMLİ: Her park yeri için sol üst köşeden sağ alt köşeye doğru
        fareyi sürükleyerek tam alanı kaplayacak şekilde çizin!

✓ Park yeri eklendi: #1 - Konum:(391,231) Boyut:[63x30]
✓ Park yeri eklendi: #2 - Konum:(323,81) Boyut:[64x37]
✓ Park yeri eklendi: #3 - Konum:(398,124) Boyut:[59x29]
✓ Park yeri eklendi: #4 - Konum:(399,89) Boyut:[63x22]
✓ Park yeri eklendi: #5 - Konum:(328,45) Boyut:[62x37]

KAYIT BAŞARILI!
✓ Toplam 5 park yeri kaydedildi

Boyut İstatistikleri:
  • Genişlik: Min=59p